In [1]:
import numpy as np 
import pandas as pd
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
import pickle as cPickle
from os.path import exists
from pathlib import Path
import csv
import datetime
import pytz

from pathlib import Path 
import time

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/training_data_all_20220203.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103356 entries, 0 to 103355
Columns: 300 entries, Unnamed: 0 to wb2sum
dtypes: float64(290), int64(5), object(5)
memory usage: 236.6+ MB


In [3]:
usgs_reg = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/USGS_region_lookup.csv')
usgs_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NA_L3CODE   11 non-null     object
 1   USGS_Group  11 non-null     object
dtypes: object(2)
memory usage: 304.0+ bytes


In [4]:
region_dict = dict(zip(usgs_reg.NA_L3CODE, usgs_reg.USGS_Group))
region_dict

{'10.1.5': 'Xeric',
 '10.2.1': 'Xeric',
 '10.2.2': 'Xeric',
 '11.1.1': 'Xeric',
 '11.1.2': 'Xeric',
 '11.1.3': 'Xeric',
 '6.2.11': 'North Coastal Mountains',
 '6.2.12': 'Interior Mountains',
 '6.2.7': 'Interior Mountains',
 '6.2.8': 'Interior Mountains',
 '7.1.8': 'North Coastal Mountains'}

In [5]:
data['NewEco3'] = data['NewEco3'].replace(region_dict)

In [6]:
print(data['NewEco3'].unique())

['Xeric' 'Interior Mountains' 'North Coastal Mountains']


In [7]:
records_with_missing_data = len(data[data.isna().any(axis=1)])
print('Dropping {} records with missing data'.format(records_with_missing_data))
data = data.dropna(axis=0,how='any').copy()
#data.isnull().sum().sum()
data['flow_cfs'] = data['flow_cfs'] / data['DRAIN_SQKM']


Dropping 3 records with missing data


In [8]:
data.head(2)

,Unnamed: 0,gage_id,year,month,flow_cfs,comid,statistic,variable,ECO3,NHDPLUSREG,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
0,0,10258000,1950,1,0.010533,22593497,mean,observed,8.0,18.0,...,40.085585,36.469271,13.544983,6.353392,173.076519,164.956173,145.791419,48.690715,22.244474,19.426220
1,1,10258500,1950,1,0.001901,22593537,mean,observed,8.0,18.0,...,9.309282,18.295394,20.328231,8.441266,83.834380,83.283639,71.869830,6.863135,4.810008,4.094633


In [9]:
data['year'].unique()

array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021])

In [10]:
data = data.loc[(data['year'] <=2015) & (data['NewEco3'] == 'Xeric')] 
data

,Unnamed: 0,gage_id,year,month,flow_cfs,comid,statistic,variable,ECO3,NHDPLUSREG,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
0,0,10258000,1950,1,0.010533,22593497,mean,observed,8.0,18.0,...,40.085585,36.469271,13.544983,6.353392,173.076519,164.956173,145.791419,48.690715,22.244474,19.426220
1,1,10258500,1950,1,0.001901,22593537,mean,observed,8.0,18.0,...,9.309282,18.295394,20.328231,8.441266,83.834380,83.283639,71.869830,6.863135,4.810008,4.094633
2,2,10259000,1950,1,0.095711,22592131,mean,observed,8.0,18.0,...,12.818197,26.513515,21.377368,9.551242,95.562592,93.623910,80.350174,7.734838,4.840720,3.977235
3,3,10263500,1950,1,0.085372,22684930,mean,observed,8.0,18.0,...,46.739904,55.791413,18.298641,4.255288,197.917833,196.351228,190.395853,56.538718,46.032462,44.989842
4,4,10264000,1950,1,0.035991,22681934,mean,observed,8.0,18.0,...,27.838182,36.697448,18.553784,3.334407,155.306407,154.299421,149.492604,30.426053,24.340323,23.668884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96564,62,11299600,2015,12,0.577641,348419,mean,observed,6.0,18.0,...,11.863177,27.533468,5.744899,19.796663,125.054731,125.054265,125.027081,8.995615,6.557477,6.381967
96569,67,11374000,2015,12,0.877739,2495212,mean,observed,6.0,18.0,...,42.261509,86.047169,94.607865,196.938443,135.053303,120.477959,101.149022,16.785335,6.928227,5.309756
96577,75,11449500,2015,12,0.597369,948020963,mean,observed,6.0,17.0,...,67.283428,145.351764,99.909700,222.960078,80.462215,74.023918,61.486000,20.451517,5.526574,3.856605
96578,76,11451100,2015,12,0.554340,8005975,mean,observed,6.0,18.0,...,49.595542,106.271222,85.936012,190.098361,87.378489,75.110422,55.707281,16.407046,5.078215,3.342670


In [11]:
data['year'].unique()

array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015])

In [12]:
data['NewEco3'].unique()

array(['Xeric'], dtype=object)

In [14]:
#dropping inessential columns
col = ['comid','Unnamed: 0','year','gage_id','statistic','variable','NHDPLUSREG','comidyear','ECO3','AggEcoRegion','NewEco3']
df = data.drop(columns = col)

In [15]:
df.head(2)

,month,flow_cfs,DRAIN_SQKM,SLOPE_PCT_30M,ELEV_MEAN_M_BASIN_30M,ELEV_MIN_M_BASIN_30M,ELEV_MAX_M_BASIN_30M,ElvRng,TOT_ECOL3_1,TOT_ECOL3_2,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
0,1,0.010533,44.1009,45.96,2066.05,171.75,3264.80,3093.05,0.0,0.0,...,40.085585,36.469271,13.544983,6.353392,173.076519,164.956173,145.791419,48.690715,22.244474,19.426220
1,1,0.001901,244.3446,28.19,1186.88,189.20,2453.58,2264.38,0.0,0.0,...,9.309282,18.295394,20.328231,8.441266,83.834380,83.283639,71.869830,6.863135,4.810008,4.094633


In [16]:
df.columns

Index(['month', 'flow_cfs', 'DRAIN_SQKM', 'SLOPE_PCT_30M',
       'ELEV_MEAN_M_BASIN_30M', 'ELEV_MIN_M_BASIN_30M', 'ELEV_MAX_M_BASIN_30M',
       'ElvRng', 'TOT_ECOL3_1', 'TOT_ECOL3_2',
       ...
       'wb9', 'wb10', 'wb11', 'wb12', 'p6sum', 'p3sum', 'p2sum', 'wb6sum',
       'wb3sum', 'wb2sum'],
      dtype='object', length=289)

In [17]:
df[df['month']==11].head()

,month,flow_cfs,DRAIN_SQKM,SLOPE_PCT_30M,ELEV_MEAN_M_BASIN_30M,ELEV_MIN_M_BASIN_30M,ELEV_MAX_M_BASIN_30M,ElvRng,TOT_ECOL3_1,TOT_ECOL3_2,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
742,11,0.007940,378.6705,29.26,1186.90,370.17,2606.12,2235.95,0.0,0.0,...,5.547031,4.952814,2.049074,1.470940,40.306994,10.256358,10.243450,3.281123,0.653459,0.572447
743,11,0.000000,57.7287,44.82,1339.49,317.80,1952.06,1634.26,0.0,0.0,...,3.567352,2.615263,1.631448,1.307082,50.369502,14.508918,14.508918,3.209170,0.802190,0.758336
744,11,0.000000,44.1009,45.96,2066.05,171.75,3264.80,3093.05,0.0,0.0,...,40.238475,31.533567,15.868234,3.557986,34.978840,13.989436,12.017898,31.897589,4.049460,2.036604
745,11,0.000000,244.3446,28.19,1186.88,189.20,2453.58,2264.38,0.0,0.0,...,9.436281,7.186150,2.651338,1.443295,31.607202,11.278973,11.278928,3.706943,0.800236,0.665212
746,11,0.049523,23.1543,55.34,1336.75,177.49,2568.37,2390.88,0.0,0.0,...,15.061199,11.468925,2.323169,1.654066,26.375435,9.378821,9.281827,4.523855,0.825061,0.616714


In [22]:
def x_train(month):
  # train = df[df["comid"].isin(comid)]
  train = df[df['month']==month]
  train.reset_index(drop=True,inplace=True)
  x_train = train.drop(columns = ['flow_cfs']).copy()
  return x_train

def y_train(month):
  # train = df[df["comid"].isin(comid)]
  train = df[df['month']==month]
  train.reset_index(drop=True,inplace=True)
  y_train = train.flow_cfs.values 
  return y_train

def x_valid(month):
  # valid = df[df["comid"].isin(comid)]
  valid = df[df['month']==month]
  valid.reset_index(drop=True,inplace=True)
  x_valid = valid.drop(columns = ['flow_cfs']).copy()
  return x_valid

def y_valid(month):
  # valid = df[df["comid"].isin(comid)]
  valid = df[df['month']==month]
  valid.reset_index(drop=True,inplace=True)
  y_valid = valid.flow_cfs.values
  return y_valid

def x_test(month):  
  # test = df[df["comid"].isin(comid)]
  test = df[df['month']==month]
  test.reset_index(drop=True,inplace=True)
  x_test = test.drop(columns = ['flow_cfs']).copy()
  return x_test

def y_test(month):
  # test = df[df["comid"].isin(comid)]
  test = df[df['month']==month]
  test.reset_index(drop=True,inplace=True)  
  y_test = test.flow_cfs.values
  return y_test


In [21]:
rf = RandomForestRegressor(n_jobs = -1, n_estimators=100, min_samples_leaf=1,max_features="sqrt", max_depth=5, verbose=1) 
rf

RandomForestRegressor(max_depth=5, max_features='sqrt', n_jobs=-1, verbose=1)

In [23]:
def hyperparameter_csv_append(month, model_run_date_time, r2, leaf, features, depth, estimator):
  if exists('/content/drive/MyDrive/TNC_Collab_Results/Results from USGS RF (Xeric-all-comids) model/hyperparameter_'+str(month)+'.csv') == False:

    with open('/content/drive/MyDrive/TNC_Collab_Results/Results from USGS RF (Xeric-all-comids) model/hyperparameter_'+str(month)+'.csv', 'w') as f:
      writer = csv.writer(f)
      header = ["model_run_date_time", "R2", "leaf", "features", "depth", "estimator"]
      writer.writerow(header)
      f.close()

  with open('/content/drive/MyDrive/TNC_Collab_Results/Results from USGS RF (Xeric-all-comids) model/hyperparameter_'+str(month)+'.csv', 'a') as f:
    row = [str(model_run_date_time), str(r2), str(leaf), str(features), str(depth), str(estimator)]
    new_row = ",".join(row)
    f.write(new_row)
    f.write("\n")

def hyper_parameter_search(n_iter, month): 
 
    for i in range(n_iter):   
        # n_estimators = The number of trees in the forest. 
        estimators = [1000] #ntrees in r. Ted used 1000

        for count in estimators:
            rf = RandomForestRegressor(n_jobs = -1, n_estimators=count, min_samples_leaf=1,max_features="sqrt", max_depth=5, verbose=1) 
            # print("min_samples_leaf %d, max_features %.1f, max_depth %d, estimator %d" % (h1, h2, h3,count))
            print ("min_samples_leaf %d, max_features %.1f, max_depth %d, estimator %d" % (1, 17, 5,count))
            print (rf)
            rf.fit(x_train(month), y_train(month))
            predictions = rf.predict(x_valid(month))
            features_sqrt = int(math.sqrt(len(x_train(month).columns)))

            # Mean absolute error (MAE)
            mae = mean_absolute_error(y_valid(month), predictions)
            # Mean squared error (MSE)
            mse = mean_squared_error(y_valid(month), predictions)
            # R-squared scores 
            r2 = r2_score(y_valid(month)+1, predictions+1)

            # Print metrics
            print('Mean Absolute Error:', round(mae, 2))
            print('Mean Squared Error:', round(mse, 2))
            print('R-squared scores:', round(r2, 2))            

            model_run_date_time = datetime.datetime.now(pytz.timezone('US/Pacific'))

            hyperparameter_csv_append(month, model_run_date_time, r2, 1, features_sqrt, 5, count)  


In [26]:
def train_final_model(month):

  df_usgs_reduced = pd.read_excel('/content/drive/MyDrive/TNC_Collab_Results/USGS_reduced_variables(Zimmerman_etal_2017_Supp_info2).xlsx', sheet_name = 3) #Change the sheetname according to region, Sheet 1 = Interior_Mountains, Sheet 2 = Coastal_Mountains, Sheet 3 = Xeric
  df_usgs_reduced = df_usgs_reduced.loc[(df_usgs_reduced['Monthly.Stat'] == 'Mean') & (df_usgs_reduced['Month'] == month)] 
  reduced_var_list = df_usgs_reduced['Predictor'].unique().tolist()
  print("Number of variables for final model: {}".format(len(reduced_var_list)))
  print(reduced_var_list)

  dfp = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/Results from USGS RF (Xeric-all-comids) model/hyperparameter_'+str(month)+'.csv')
  dfp = dfp.sort_values(by="R2", ascending=False)
  
  best_leaf = dfp['leaf'].iloc[0]
  best_features = dfp['features'].iloc[0]
  best_depth = dfp['depth'].iloc[0]
  best_estimator = dfp['estimator'].iloc[0]

  reduced_columns_list = ['month']
  reduced_columns_list = reduced_columns_list + reduced_var_list

  # Train Reduced model
  rf = RandomForestRegressor(n_jobs = -1, min_samples_leaf=best_leaf,max_features=best_features, max_depth=best_depth, n_estimators=best_estimator,verbose=1) 
  rf.fit(x_train(month).loc[:, reduced_columns_list], y_train(month))
  predictions = rf.predict(x_test(month).loc[:, reduced_columns_list])
  r2 = r2_score(y_test(month)+1, predictions+1)
  model_run_date_time = datetime.datetime.now(pytz.timezone('US/Pacific'))
  print("R2 for USGS_reduced model=",r2)

  rf_pickle_file = 'USGS_model_reduced_'+str(month)+'_r2_'+str(round(r2,4))+'_'+str(model_run_date_time)+'.pkl'
  with open('/content/drive/My Drive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Xeric-all-comids)/'+rf_pickle_file, 'wb') as f:
    cPickle.dump(rf, f)

In [28]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [11]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 11
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.05
Mean Squared Error: 0.01
R-squared scores: 0.77
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.05
Mean Squared Error: 0.01
R-squared scores: 0.77
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.05
Mean Squared Error: 0.01
R-squared scores: 0.76
Training final model for month 11
Number of variables for final model: 37
['wb0', 'p0', 'p7', 'PPTAVG_BASIN', 'AnnMaxPrecip', 'WD_APR_BASIN', 'ET', 't5', 'wb3sum', 'wb2sum', 'pmpe', 'wb8', 'p2', 'p3sum', 'p8', 'wb1', 'HGB', 'wb9', 't11', 't0', 'wb2', 'p2sum', 'SLOPE_PCT_30M', 'p10', 't1', 'p1', 'RECHARGE', 'WD_DEC_BASIN', 'HLR18', 'p6', 't2', 'PPTAVG_CAT', 'CAT_AnnMaxPrecip', 'WDMAX_BASIN', 'WD_MAR_BASIN', 'WD_FEB_BASIN', 'HLR16']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   11.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s


R2 for USGS_reduced model= 0.931042492881777


[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.4s finished


In [29]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [12]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 12
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    7.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.13
Mean Squared Error: 0.07
R-squared scores: 0.76
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.13
Mean Squared Error: 0.07
R-squared scores: 0.75
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


...

Mean Absolute Error: 0.13
Mean Squared Error: 0.07
R-squared scores: 0.75
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.13
Mean Squared Error: 0.07
R-squared scores: 0.76
Training final model for month 12
Number of variables for final model: 28
['wb0', 'p0', 'WD_APR_BASIN', 'PPTAVG_BASIN', 'ET', 'WD_DEC_BASIN', 'p1', 'p2sum', 'PPTAVG_CAT', 'p7', 'wb9', 'wb1', 'p3sum', 'WD_NOV_BASIN', 'wb12', 'wb2sum', 'p6sum', 'HLR16', 'PERDUN', 'WDMAX_BASIN', 'ANN_CONS_WETDAYS', 'pmpe', 'WD_BASIN', 'WD_FEB_BASIN', 'RECHARGE', 'WD_JAN_BASIN', 'CAT_AnnMaxPrecip', 'AnnMaxPrecip']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    9.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_reduced model= 0.9174757848967174


In [30]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [1]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 1
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.28
Mean Squared Error: 0.32
R-squared scores: 0.69
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.28
Mean Squared Error: 0.32
R-squared scores: 0.69
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


...

Mean Absolute Error: 0.28
Mean Squared Error: 0.31
R-squared scores: 0.69
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.28
Mean Squared Error: 0.32
R-squared scores: 0.69
Training final model for month 1
Number of variables for final model: 33
['wb0', 'p0', 'WD_APR_BASIN', 'ET', 'WD_DEC_BASIN', 'p3sum', 'p6sum', 'wb1', 't10', 'p3', 'p1', 'WD_NOV_BASIN', 't0', 't11', 'RH_BASIN', 'wb3sum', 't2', 'p2sum', 'wb2sum', 'PPTAVG_BASIN', 'ELEV_MAX_M_BASIN_30M', 'HLR20', 'WD_FEB_BASIN', 'AnnMaxPrecip', 'TOPWET', 'WD_MAR_BASIN', 'WDMAX_BASIN', 'WD_BASIN', 'WD_JAN_BASIN', 'wb6sum', 'RECHARGE', 'PPTAVG_CAT', 'pmpe']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    9.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_reduced model= 0.9127089730300804


In [31]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [2]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 2
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.34
Mean Squared Error: 0.38
R-squared scores: 0.71
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.33
Mean Squared Error: 0.38
R-squared scores: 0.71
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


...

Mean Absolute Error: 0.33
Mean Squared Error: 0.39
R-squared scores: 0.71
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.34
Mean Squared Error: 0.39
R-squared scores: 0.7
Training final model for month 2
Number of variables for final model: 34
['wb0', 'p0', 'wb1', 'p6sum', 'p1', 't1', 'p2sum', 'WD_APR_BASIN', 'WD_DEC_BASIN', 'p3sum', 'wb2sum', 'ELEV_MAX_M_BASIN_30M', 'RECHARGE', 'ELEV_MEAN_M_BASIN_30M', 'p12', 'p10', 'p11', 't9', 'ET', 't3', 'wb3sum', 'PERMAVE', 'p5', 'p9', 't5', 'WD_NOV_BASIN', 'WD_FEB_BASIN', 'p3', 'wb6sum', 'WD_JAN_BASIN', 'ET_CAT', 'PERDUN', 'SNOW_PCT_PRECIP', 'ElvRng']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   11.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_reduced model= 0.9107550266576224


In [32]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [3]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 3
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.21
Mean Squared Error: 0.15
R-squared scores: 0.76
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.21
Mean Squared Error: 0.15
R-squared scores: 0.76
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


...

Mean Absolute Error: 0.21
Mean Squared Error: 0.14
R-squared scores: 0.77
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.21
Mean Squared Error: 0.15
R-squared scores: 0.77
Training final model for month 3
Number of variables for final model: 29
['wb0', 'p0', 'wb2sum', 'wb1', 'p6sum', 'p3sum', 'wb3sum', 'RECHARGE', 'p11', 'WD_APR_BASIN', 'wb6sum', 'WD_DEC_BASIN', 'p10', 'WD_NOV_BASIN', 'p1', 'p2sum', 't4', 'RH_BASIN', 'BDAVE', 't0', 'WD_BASIN', 'p12', 'p2', 'wb3', 't6', 'p4', 'wb2', 'wb7', 'PET_CAT']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   12.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_reduced model= 0.8947484279385988


In [33]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [4]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 4
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.12
Mean Squared Error: 0.06
R-squared scores: 0.76
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.12
Mean Squared Error: 0.06
R-squared scores: 0.77
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


...

Mean Absolute Error: 0.12
Mean Squared Error: 0.06
R-squared scores: 0.77
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.12
Mean Squared Error: 0.06
R-squared scores: 0.77
Training final model for month 4
Number of variables for final model: 25
['wb0', 'p0', 'wb1', 'p6sum', 'wb6sum', 'wb3sum', 'wb2sum', 'WD_FEB_BASIN', 'RECHARGE', 'p1', 't6', 't10', 'p3sum', 't8', 'p6', 'wb5', 't0', 'HLR20', 't1', 'wb6', 'wb4', 'p11', 'wb2', 'p2sum', 'PET_CAT']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   12.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_reduced model= 0.8854668982312659


In [34]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [5]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 5
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.05
Mean Squared Error: 0.01
R-squared scores: 0.82
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.05
Mean Squared Error: 0.01
R-squared scores: 0.82
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


...

Mean Absolute Error: 0.05
Mean Squared Error: 0.01
R-squared scores: 0.82
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.05
Mean Squared Error: 0.01
R-squared scores: 0.82
Training final model for month 5
Number of variables for final model: 46
['wb0', 'p6sum', 'wb1', 'wb6sum', 'wb2sum', 'RECHARGE', 't1', 'wb3sum', 'ELEV_MAX_M_BASIN_30M', 'T_MAX_BASIN', 'AWCAVE', 'KFACT_UP', 'NO200AVE', 'p0', 'p2sum', 't0', 'p3sum', 'T_AVG_CAT', 'PET_CAT', 'wb2', 'HLR20', 'NO4AVE', 'wb4', 'SLOPE_PCT_30M', 'NO10AVE', 'p1', 'WD_SEP_BASIN', 'LST32F_BASIN', 't12', 't11', 'DEPTH_WATTAB', 'PERHOR', 'ELEV_MEAN_M_BASIN_30M', 'WD_JUL_BASIN', 'SNOW_PCT_PRECIP', 't7', 'pmpe', 'PET', 'PERMAVE', 'AnnMaxPrecip', 'BDAVE', 'RFACT', 'WD_BASIN', 'WD_OCT_BASIN', 'WD_FEB_BASIN', 'WD_NOV_BASIN']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    9.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_reduced model= 0.8865725306190174


In [35]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [6]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 6
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.03
Mean Squared Error: 0.0
R-squared scores: 0.88
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.03
Mean Squared Error: 0.0
R-squared scores: 0.88
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


...

Mean Absolute Error: 0.03
Mean Squared Error: 0.0
R-squared scores: 0.88
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.03
Mean Squared Error: 0.0
R-squared scores: 0.88
Training final model for month 6
Number of variables for final model: 54
['wb0', 'SNOW_PCT_PRECIP', 'WD_SEP_BASIN', 'wb1', 'RECHARGE', 'p6sum', 'wb6sum', 'WD_JUL_BASIN', 'LST32F_BASIN', 'WDMIN_BASIN', 'wb2sum', 'wb3sum', 'SLOPE_PCT_30M', 'NO4AVE', 'NO200AVE', 'ELEV_MAX_M_BASIN_30M', 'T_MAX_BASIN', 't2', 'wb5', 'AWCAVE', 'DEPTH_WATTAB', 'wb2', 'NO10AVE', 'KFACT_UP', 'HLR20', 't0', 'TOT_ECOL3_5', 'p1', 't1', 'RFACT', 'ELEV_MEAN_M_BASIN_30M', 't11', 'T_AVG_CAT', 'PET_CAT', 't9', 't3', 'CONTACT', 't4', 't6', 't10', 'ELEV_MIN_M_BASIN_30M', 'T_MIN_BASIN', 'PERHOR', 't8', 'FST32F_BASIN', 'BDAVE', 'PERMAVE', 'AnnMinPrecip', 'FeO_pct', 'HGB', 'WD_OCT_BASIN', 'WD_MAY_BASIN', 'CAT_AnnMinPrecip', 'SGEO20']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    8.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_reduced model= 0.9157461527834936


In [36]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [7]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 7
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.86
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.86
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


...

Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.86
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.86
Training final model for month 7
Number of variables for final model: 51
['wb1', 'wb0', 'wb2sum', 'p0', 't1', 'TOT_ECOL3_5', 'SLOPE_PCT_30M', 'wb6sum', 'FST32F_BASIN', 'PET_CAT', 'LST32F_BASIN', 'p6sum', 'WDMIN_BASIN', 'wb2', 'HLR20', 'T_MIN_BASIN', 'AWCAVE', 'p12', 'T_AVG_CAT', 'WD_JUL_BASIN', 'wb3sum', 'NO10AVE', 'p6', 'RFACT', 'NO4AVE', 'DEPTH_WATTAB', 'T_MAX_BASIN', 'p11', 'wb4', 't3', 'CONTACT', 't2', 'wb3', 't7', 't5', 't12', 't0', 't4', 'RECHARGE', 't10', 'PET', 'ELEV_MIN_M_BASIN_30M', 'SNOW_PCT_PRECIP', 'ELEV_MEAN_M_BASIN_30M', 'BDAVE', 'WD_OCT_BASIN', 'HGB', 'FeO_pct', 'WD_SEP_BASIN', 'WD_MAY_BASIN', 'SGEO20']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    9.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_reduced model= 0.8911575576304687


In [37]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [8]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 8
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.01
Mean Squared Error: 0.0
R-squared scores: 0.71
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.01
Mean Squared Error: 0.0
R-squared scores: 0.71
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


...

Mean Absolute Error: 0.01
Mean Squared Error: 0.0
R-squared scores: 0.71
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.01
Mean Squared Error: 0.0
R-squared scores: 0.71
Training final model for month 8
Number of variables for final model: 41
['wb0', 'p0', 'wb1', 'NO10AVE', 'wb3sum', 'wb2sum', 'NO4AVE', 'wb2', 'wb6sum', 'SLOPE_PCT_30M', 'RFACT', 'DEPTH_WATTAB', 'p12', 'wb7', 'HLR20', 'wb3', 'p1', 'ELEV_MIN_M_BASIN_30M', 't10', 't2', 'p2sum', 'wb5', 't7', 't1', 'T_AVG_CAT', 'wb6', 't12', 'CONTACT', 'AWCAVE', 'p4', 't8', 'T_MAX_BASIN', 'PET_CAT', 'KFACT_UP', 'ELEV_MEAN_M_BASIN_30M', 'AnnMaxPrecip', 'PET', 'RECHARGE', 'SGEO20', 'WD_JUL_BASIN', 'ELEV_MAX_M_BASIN_30M']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    9.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_reduced model= 0.7445843355495618


In [38]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [9]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 9
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.01
Mean Squared Error: 0.0
R-squared scores: 0.72
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.01
Mean Squared Error: 0.0
R-squared scores: 0.72
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


...

Mean Absolute Error: 0.01
Mean Squared Error: 0.0
R-squared scores: 0.72
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.01
Mean Squared Error: 0.0
R-squared scores: 0.72
Training final model for month 9
Number of variables for final model: 37
['SLOPE_PCT_30M', 'wb0', 'RFACT', 'wb2', 'wb1', 'NO10AVE', 'wb2sum', 'wb6sum', 'wb3sum', 'HLR20', 'p0', 'wb3', 'AWCAVE', 'wb6', 'wb7', 'p8', 't3', 'NO4AVE', 'wb4', 'CONTACT', 't6', 'DEPTH_WATTAB', 't12', 'wb5', 'NO200AVE', 'PET_CAT', 'AnnMaxPrecip', 'T_MAX_BASIN', 'T_AVG_CAT', 'PET', 'RECHARGE', 'KFACT_UP', 'T_AVG_BASIN', 'SNOW_PCT_PRECIP', 'ELEV_MEAN_M_BASIN_30M', 'pmpe', 'ELEV_MAX_M_BASIN_30M']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    9.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_reduced model= 0.7813008301840072


In [39]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [10]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 10
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.68
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.69
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


...

Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.68
min_samples_leaf 1, max_features 17.0, max_depth 5, estimator 1000
RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=1000,
                      n_jobs=-1, verbose=1)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    6.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.02
Mean Squared Error: 0.0
R-squared scores: 0.68
Training final model for month 10
Number of variables for final model: 44
['p0', 'wb0', 'pmpe', 'SLOPE_PCT_30M', 'wb2', 'wb8', 'p8', 'NO10AVE', 'AnnMaxPrecip', 'wb7', 'wb1', 'RECHARGE', 'RFACT', 'p1', 't9', 'wb2sum', 't5', 'wb3sum', 'p10', 'wb9', 'wb4', 'wb10', 'PPTAVG_BASIN', 'NO4AVE', 'p2sum', 'wb6sum', 'wb3', 'TOT_ECOL3_14', 't6', 't7', 'WD_FEB_BASIN', 'CONTACT', 'HLR20', 'KFACT_UP', 'p3sum', 'NO200AVE', 'PET_CAT', 'ELEV_MIN_M_BASIN_30M', 'AWCAVE', 'WD_BASIN', 'WD_JAN_BASIN', 'ELEV_MAX_M_BASIN_30M', 'WDMAX_BASIN', 'T_AVG_CAT']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   10.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_reduced model= 0.8540160760480849
